## Customer Lifetime Value ~ CLTV 

**It is the present value of the future cash flows or the value of business attributed to the customer during his or her entire relationship with the company. Description: CLTV is the value a customer contributes to your business over the entire lifetime at your company.**

In [1]:
! pip install openpyxl
! pip install lifetimes

     |████████████████████████████████| 243 kB 282 kB/s 
     |████████████████████████████████| 584 kB 404 kB/s 
  Created wheel for autograd: filename=autograd-1.3-py3-none-any.whl size=47989 sha256=9850118ce641e6e75e1c0dd73ea91f217561d183d60d6acbad2ae9bb89901b27
  Stored in directory: /root/.cache/pip/wheels/ef/32/31/0e87227cd0ca1d99ad51fbe4b54c6fa02afccf7e483d045e04
Successfully built autograd


In [2]:
import datetime as dt
import pandas as pd
import matplotlib.pyplot as plt
from lifetimes import BetaGeoFitter
from lifetimes import GammaGammaFitter
from lifetimes.plotting import plot_period_transactions
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [3]:
df = pd.read_excel('/kaggle/input/online-retail/online_retail.xlsx', sheet_name="Year 2010-2011", engine='openpyxl')

In [4]:
# Analysis for United Kingdom

df=df[df["Country"]=="United Kingdom"]

In [5]:
#########################
# Pre Processing
#########################

def outlier_thresholds(dataframe, variable):
    quartile1 = dataframe[variable].quantile(0.01)
    quartile3 = dataframe[variable].quantile(0.99)
    interquantile_range = quartile3 - quartile1
    up_limit = quartile3 + 1.5 * interquantile_range
    low_limit = quartile1 - 1.5 * interquantile_range
    return low_limit, up_limit


def replace_with_thresholds(dataframe, variable):
    low_limit, up_limit = outlier_thresholds(dataframe, variable)
    dataframe.loc[(dataframe[variable] < low_limit), variable] = low_limit
    dataframe.loc[(dataframe[variable] > up_limit), variable] = up_limit

df.dropna(inplace=True)
df = df[~df["Invoice"].str.contains("C", na=False)]
df = df[df["Quantity"] > 0]

replace_with_thresholds(df, "Quantity")
replace_with_thresholds(df, "Price")
df.describe().T

df["TotalPrice"] = df["Quantity"] * df["Price"]

today_date = dt.datetime(2011, 12, 11)

df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,TotalPrice
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6.0,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6.0,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8.0,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6.0,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6.0,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34


In [6]:
#########################
# Preparing Dataset
#########################

# Recency: How recently a customer has made a purchase
# Frequency: How often a customer makes a purchase
# Monetary Value: How much money a customer spends on purchases

cltv_df = df.groupby('Customer ID').agg({'InvoiceDate': [lambda date: (date.max() - date.min()).days,
                                                         lambda date: (today_date - date.min()).days],
                                        'Invoice': lambda num: num.nunique(),
                                        'TotalPrice': lambda TotalPrice: TotalPrice.sum()})

cltv_df.columns = cltv_df.columns.droplevel(0)
cltv_df.columns = ['recency', 'T', 'frequency', 'monetary']

cltv_df["monetary"] = cltv_df["monetary"] / cltv_df["frequency"]

cltv_df = cltv_df[cltv_df["monetary"] > 0]

cltv_df.head()

,recency,T,frequency,monetary
Customer ID,,,,
12346.0,0,326,1,258.440000
12747.0,366,370,11,381.455455
12748.0,372,374,210,153.828143
12749.0,209,214,5,814.488000
12820.0,323,327,4,235.585000


In [7]:
# Eecency and T value is converted to "Weekly" 
cltv_df["recency"] = cltv_df["recency"] / 7
cltv_df["T"] = cltv_df["T"] / 7

In [8]:
# If there is lower 1 transaction value, then they are not selected: 
cltv_df = cltv_df[(cltv_df['frequency'] > 1)]

In [9]:
bgf = BetaGeoFitter(penalizer_coef=0.001)

bgf.fit(cltv_df['frequency'],
        cltv_df['recency'],
        cltv_df['T'])

# 1 haftalık beklenen transaction
cltv_df["expected_purc_1_week"] = bgf.predict(1,
                                              cltv_df['frequency'],
                                              cltv_df['recency'],
                                              cltv_df['T'])

In [10]:
# For one Monthly, Expcexted Purchase Transactions count
cltv_df["expected_purc_1_month"] = bgf.predict(4,
                                               cltv_df['frequency'],
                                               cltv_df['recency'],
                                               cltv_df['T'])

In [11]:
# Expected_average_profit for one customer:

ggf = GammaGammaFitter(penalizer_coef=0.01)
ggf.fit(cltv_df['frequency'], cltv_df['monetary'])

cltv_df["expected_average_profit"] = ggf.conditional_expected_average_profit(cltv_df['frequency'],
                                                                             cltv_df['monetary'])

cltv_df.head()

,recency,T,frequency,monetary,expected_purc_1_week,expected_purc_1_month,expected_average_profit
Customer ID,,,,,,,
12747.0,52.285714,52.857143,11,381.455455,0.202479,0.807679,387.822846
12748.0,53.142857,53.428571,210,153.828143,3.252952,12.977515,153.971325
12749.0,29.857143,30.571429,5,814.488000,0.167149,0.665729,844.094697
12820.0,46.142857,46.714286,4,235.585000,0.103972,0.414566,247.080949
12822.0,2.285714,12.571429,2,474.440000,0.129141,0.512653,520.828195


In [12]:
# Customer Lifetime Value: for 6 Month:
cltv_6 = ggf.customer_lifetime_value(bgf,
                                   cltv_df['frequency'],
                                   cltv_df['recency'],
                                   cltv_df['T'],
                                   cltv_df['monetary'],
                                   time=6,  # 6 Month
                                   freq="W",  # T'nin frekans bilgisi.
                                   discount_rate=0.01)

cltv_6 = cltv_6.reset_index()

In [13]:
# Customer Lifetime Value:
cltv_6.head()

,Customer ID,clv
0,12747.0,1937.046140
1,12748.0,12365.796180
2,12749.0,3446.010436
3,12820.0,631.939332
4,12822.0,1612.096652
